In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
from datetime import datetime, timedelta
from typing import Any, List

import numpy as np
import pandas as pd
import pandera as pa
import pytz
from ib_insync import (
    IB,
    Contract,
    FuturesOption,
    Index,
    Ticker,
    util,
)
from loguru import logger
from loman import ComputationFactory, input_node, calc_node
from tqdm import tqdm

from ib_insync_options.etl.ibkr_etl import map_ticker_to_exchange, MarketDataType
from ib_insync_options.utils.date_utils import calc_effective_date_of_dt
from ib_insync_options.utils.dict_utils import gen_json
from ib_insync_options.utils.formatting_utils import lower_camel_case_to_snake_case
from ib_insync_options.utils.list_utils import chunks
from ib_insync_options.utils.networking import get_ibkr_host_ip
from ib_insync_options.etl.facets import FacetCoreDfColumns, CoreFacetSchema

import pytz
from ib_insync_options.utils.date_utils import calc_effective_date_of_dt
from ib_insync_options.etl.instrument_multiplexer import InstrumentMultiplexer

In [3]:
# required for jupyter notebook
util.startLoop()
ib = IB()

host = get_ibkr_host_ip()
ib.connect(host, 7496, clientId=3, timeout=30)

<IB connected to 127.0.0.1:7496 clientId=3>

In [4]:
def default_effective_date():
    utc_now = datetime.now(pytz.utc)
    mt_now = utc_now.astimezone(pytz.timezone("US/Mountain"))
    dt = calc_effective_date_of_dt(mt_now)
    return datetime(dt.year, dt.month, dt.day, tzinfo=pytz.timezone("US/Mountain"))

In [5]:
@ComputationFactory
class IbkrFutureOptionData:
    # nodes with default values
    asset_category = input_node(value="future")
    dte_max = input_node(value=30)
    effective_date = input_node(lambda: default_effective_date())
    # market_data_type = input_node(value=MarketDataType.LIVE)

    # input nodes without default values
    raw_option_chain = input_node()
    pre_fetch_option_filters = input_node()
    qualified_contracts = input_node()
    underlying_df = input_node()
    underlying_symbol = input_node()
    enriched_option_chain_df = input_node()

    @calc_node
    def expiry_date_max(effective_date, dte_max):
        dt = effective_date + timedelta(days=dte_max)
        return datetime(dt.year, dt.month, dt.day, tzinfo=pytz.timezone("US/Mountain"))

    @calc_node
    def option_chain_df(raw_option_chain, underlying_symbol):
        df = raw_option_chain.copy()

        df = df.explode("expirations")
        df = df.explode("strikes")
        df = df.rename(columns={"expirations": "expiry", "strikes": "strike"})
        df["expiry"] = pd.to_datetime(df["expiry"], format="%Y%m%d", errors="coerce")

        df = df.rename(columns=lower_camel_case_to_snake_case)

        df.strike = df.strike.astype("float64")

        df = df.sort_values(by=["expiry", "strike"])
        df = df.reset_index(drop=True)

        df.underlying_con_id = df.underlying_con_id.astype(int)

        # remove the EC{symbol} contracts
        df = df[~df.trading_class.str.startswith("EC")]

        # set underlying_symbol
        df["underlying_symbol"] = underlying_symbol

        return df

    @calc_node
    def filter_option_chain_by_max_expiry_date(option_chain_df, expiry_date_max):
        df = option_chain_df.copy()
        max_dt = pd.to_datetime(expiry_date_max)
        # only keep options expiring before the max date
        df = df[pd.to_datetime(df["expiry"]).dt.date <= max_dt.date()]
        return df

    @calc_node
    def underlying_df_calc_price(underlying_df):
        df = underlying_df.copy()

        # if bid AND ask are -1.0, use last
        mask_bid_ask_negative_one = (df["bid"] == -1.0) & (df["ask"] == -1.0)
        df.loc[mask_bid_ask_negative_one, "finx_price"] = df.loc[
            mask_bid_ask_negative_one, "last"
        ]

        # if bid OR ask are missing, use last
        mask_bid_ask_missing = df[["bid", "ask"]].isna().any(axis=1)
        df.loc[mask_bid_ask_missing, "finx_price"] = df.loc[
            mask_bid_ask_missing, "last"
        ]

        # if bid and ask have non-missing values, use the average
        mask_bid_ask_present = ~mask_bid_ask_missing & ~mask_bid_ask_negative_one
        df.loc[mask_bid_ask_present, "finx_price"] = (
            df.loc[mask_bid_ask_present, "bid"] + df.loc[mask_bid_ask_present, "ask"]
        ) / 2

        # if finx_price is nan, use close
        mask_finx_price_nan = df["finx_price"].isna()
        df.loc[mask_finx_price_nan, "finx_price"] = df.loc[mask_finx_price_nan, "close"]

        return df

    @calc_node
    def enrich_option_chain_df_with_underlying_tickers(
        filter_option_chain_by_max_expiry_date, underlying_df_calc_price
    ):
        _option_chain_df = filter_option_chain_by_max_expiry_date.copy()
        _underlying_df = underlying_df_calc_price.copy()
        mdf = pd.merge(
            left=_option_chain_df,
            right=_underlying_df[["con_id", "finx_price"]],
            left_on="underlying_con_id",
            right_on="con_id",
            how="outer",
        )

        # calc moneyness and round to 2 decimal places
        mdf["finx_moneyness"] = round((mdf.strike / mdf.finx_price) * 100.0, 2)

        return mdf

    @calc_node
    def apply_fetch_filters(
        enrich_option_chain_df_with_underlying_tickers, pre_fetch_option_filters
    ):
        df = enrich_option_chain_df_with_underlying_tickers.copy()
        _original_len = len(enrich_option_chain_df_with_underlying_tickers)

        print("")  # get logger.debug to print
        logger.debug(f"Applying pre fetch option filters: {pre_fetch_option_filters}")
        logger.debug(f"Pre Filters,  len => {len(df)}")

        for _filter in pre_fetch_option_filters:
            for key, value in _filter.items():
                if key == "moneyness_gte":
                    df = df[df["finx_moneyness"] >= value]
                elif key == "moneyness_lte":
                    df = df[df["finx_moneyness"] <= value]
                elif key == "strike_modulus_eq":
                    df = df[df["strike"] % value == 0]

        _post_filter_len = len(df)
        _percent_reduced = (_original_len - _post_filter_len) / _original_len * 100
        logger.debug(
            f"Post Filters, len => {_post_filter_len} ({_percent_reduced:.0f}% reduction)"
        )

        return df

    @calc_node
    # apply fetch filters as an arg so we know the order of operations
    def gen_qualified_fops_df(
        qualified_futures_options, apply_fetch_filters, option_chain_df
    ):
        qualified_futures_options_json = gen_json(qualified_futures_options)

        df = pd.DataFrame(qualified_futures_options_json)
        df = df.rename(columns=lower_camel_case_to_snake_case)
        df = df.rename(columns={"last_trade_date_or_contract_month": "expiry"})
        df["expiry"] = pd.to_datetime(df["expiry"])

        df = df.drop(
            columns=[
                "sec_id_type",
                "sec_id",
                "description",
                "issuer_id",
                "combo_legs_descrip",
                "combo_legs",
                "delta_neutral_contract",
                "include_expired",
                "primary_exchange",
                "sec_type",
            ]
        )

        mdf = pd.merge(
            left=option_chain_df.set_index(["expiry", "trading_class", "strike"]),
            right=df.set_index(["expiry", "trading_class", "strike"])[["con_id"]],
            left_index=True,
            right_index=True,
            how="inner",
        )
        mdf = mdf.reset_index()
        return mdf

    @calc_node
    def enrich_option_tickers_df_with_price(
        option_tickers_df, gen_qualified_fops_df, underlying_df_calc_price
    ):
        # left outer join the underlying_con_id via con_id
        mdf = pd.merge(
            left=option_tickers_df,
            right=gen_qualified_fops_df[["con_id", "underlying_con_id"]],
            left_on="source_id",
            right_on="con_id",
            how="outer",
        )
        mdf = mdf.rename(columns={"underlying_con_id": "source_underlying_id"})
        mdf = mdf.drop(columns=["con_id"])

        # left outer join the underlying price (via finx_price) via con_id
        mmdf = pd.merge(
            left=mdf,
            right=underlying_df_calc_price[["con_id", "finx_price"]],
            left_on="source_underlying_id",
            right_on="con_id",
            how="outer",
        )

        # set underlying_price to finx_price where underlying_price is None
        mmdf["underlying_price"] = mmdf["underlying_price"].fillna(mmdf["finx_price"])

        # drop intermediate and unused columns
        mmdf = mmdf.drop(columns=["finx_price", "con_id"])

        return mmdf


class IbkrDownloader:
    """
    Service Client for interacting with IBKR's TWS API.
    Specifically, downloading future option data.
    """

    def __init__(self, ib, market_data_type=MarketDataType.LIVE):
        self.ib = ib
        self.market_data_type = market_data_type

    def qualify_contracts(self, contracts: List[Contract]) -> List[Contract]:
        request_chunk_size = 500
        contract_chunks = [x for x in chunks(contracts, request_chunk_size)]

        tqdm_desc = f"ibkr=qualify-contracts-{len(contracts)}"

        qualified_contracts = []
        for chunk in tqdm(contract_chunks, desc=tqdm_desc):
            self.ib.qualifyContracts(*chunk)
            qualified_contracts.extend(chunk)

        return qualified_contracts

    def fetch_option_chain_for_underlying(
        self, underlying_symbol, sec_type: str = "FUT"
    ):
        # initialize the Index for the underlying symbol. For example, ES => ES Index
        exchange = map_ticker_to_exchange(underlying_symbol)
        index = Index(underlying_symbol, exchange, "USD")
        self.ib.qualifyContracts(index)

        # fetch the option chain
        raw_option_chain = self.ib.reqSecDefOptParams(
            underlying_symbol, "SMART", "IND", index.conId
        )
        df = util.df(raw_option_chain)
        return df

    def fetch_tickers_for_contracts(
        self, contracts: List[Contract], request_chunk_size: int = 500
    ) -> List[Any]:
        """
        Fetches tickers for a list of contracts.

        Args:
            ib (IB): ib_insync IB object
            contracts (List[Contract]): list of IB contract objects
            market_data_type (int, optional): market data type. Defaults to 1.
                Details here https://interactivebrokers.github.io/tws-api/market_data_type.html

        Returns:
            List[Any]: list of ib_insync Ticker objects

        """
        all_tickers = []
        self.ib.reqMarketDataType(self.market_data_type.value)

        contract_chunks = [x for x in chunks(contracts, request_chunk_size)]
        tqdm_desc = f"ibkr=fetch-tickers-{len(contracts)}"

        for contracts_chunk in tqdm(contract_chunks, desc=tqdm_desc):
            tickers = self.ib.reqTickers(*contracts_chunk)
            all_tickers.extend(tickers)

        return all_tickers

    def _transform_future_tickers_to_df(self, tickers):
        def transform_ticker(ticker):
            return {
                "con_id": ticker.contract.conId,
                "symbol": ticker.contract.symbol,
                "local_symbol": ticker.contract.localSymbol,
                "last_trade_date_or_contract_month": ticker.contract.lastTradeDateOrContractMonth,
                "multiplier": ticker.contract.multiplier,
                "exchange": ticker.contract.exchange,
                "currency": ticker.contract.currency,
                "last": ticker.last,
                "bid": ticker.bid,
                "ask": ticker.ask,
                "high": ticker.high,
                "low": ticker.low,
                "close": ticker.close,
                "volume": ticker.volume,
                "open_interest": ticker.futuresOpenInterest,
            }

        ticker_dicts = [transform_ticker(ticker) for ticker in tickers]

        df = pd.DataFrame(ticker_dicts)
        df.con_id = df.con_id.astype(int)
        return df

    def gen_futures_tickers_df_for_option_chain(
        self, option_chain_df: pd.DataFrame
    ) -> pd.DataFrame:
        contracts = []

        unique_underlying_con_ids = list(option_chain_df["underlying_con_id"].unique())

        # create a contract for each underlying_con_id in the option_chain_df
        for con_id in unique_underlying_con_ids:
            contract = Contract(conId=con_id)
            contracts.append(contract)

        future_contracts = self.qualify_contracts(contracts)
        tickers = self.fetch_tickers_for_contracts(future_contracts)
        df = self._transform_future_tickers_to_df(tickers)

        return df

    def fetch_underlying_df(self, option_chain_df: pd.DataFrame) -> pd.DataFrame:
        underlying_df = self.gen_futures_tickers_df_for_option_chain(option_chain_df)
        return underlying_df

    def gen_qualified_futures_options(
        self, option_chain_df: pd.DataFrame, request_chunk_size: int = 1000
    ) -> List[FuturesOption]:
        """
        Generates a list of IB qualified FuturesOption objects from a DataFrame.

        Args:
            ib (IB): ib_insync IB object
            df (pd.DataFrame): df with columns: [exchange, underlying_con_id, trading_class, multiplier, expiry, strike]
            request_chunk_size (int, optional): number of contracts to qualify at a time. Defaults to 1200.

        Returns:
            List[FuturesOption]: list of FuturesOption objects

        """
        df = option_chain_df.copy()

        options = []
        for _, row in df.iterrows():
            trade_date = row["expiry"].strftime("%Y%m%d")
            symbol = row["underlying_symbol"]
            strike = float(row["strike"])
            exchange = row["exchange"]
            multiplier = row["multiplier"]

            call = FuturesOption(
                symbol=symbol,
                lastTradeDateOrContractMonth=trade_date,
                strike=strike,
                right="C",
                exchange=exchange,
                multiplier=multiplier,
                currency="USD",
                tradingClass=row["trading_class"],
            )
            options.append(call)

            put = FuturesOption(
                symbol=symbol,
                lastTradeDateOrContractMonth=trade_date,
                strike=strike,
                right="P",
                exchange=exchange,
                multiplier=multiplier,
                currency="USD",
                tradingClass=row["trading_class"],
            )
            options.append(put)

        if len(options) > 1501:
            logger.info(
                f"Qualifying a large number of options (number = {len(options)}). This may take a long time."
            )

        contract_chunks = [x for x in chunks(options, request_chunk_size)]
        tqdm_desc = f"ibkr=fetch-future-options-{len(options)}"
        qualified_options = []
        for chunk in tqdm(contract_chunks, desc=tqdm_desc):
            self.ib.qualifyContracts(*chunk)
            qualified_options.extend(chunk)

        return qualified_options

    def gen_option_tickers_df(
        self, tickers: List[Ticker]
    ) -> pa.typing.DataFrame[CoreFacetSchema]:
        """
        Generates a DataFrame from a list of Ticker objects.

        Args:
            tickers (List[Ticker]): ib_insync Ticker objects

        Returns:
            pd.DataFrame: df

        """
        # observation date for tickers if none is provided
        utc_now = datetime.now(pytz.utc)
        et_now = utc_now.astimezone(pytz.timezone("US/Eastern"))
        mt_now = utc_now.astimezone(pytz.timezone("US/Mountain"))
        effective_date = calc_effective_date_of_dt(et_now)

        rows = []
        for ticker in tickers:
            if ticker.contract is None:
                continue

            if ticker.contract.lastTradeDateOrContractMonth is None:
                continue

            row = {}
            row[FacetCoreDfColumns.EFFECTIVE_DATETIME.value] = mt_now
            row["symbol"] = ticker.contract.symbol
            row["last"] = ticker.marketPrice()
            row[FacetCoreDfColumns.BID.value] = ticker.bid
            row[FacetCoreDfColumns.ASK.value] = ticker.ask
            row[FacetCoreDfColumns.STRIKE.value] = ticker.contract.strike

            # NOTE the date varies between YYYYMMDD or YYYY-MM-DD
            strptime_format = (
                "%Y-%m-%d"
                if "-" in ticker.contract.lastTradeDateOrContractMonth
                else "%Y%m%d"
            )
            expiry_dt = datetime.strptime(
                ticker.contract.lastTradeDateOrContractMonth, strptime_format
            )
            row[FacetCoreDfColumns.EXPIRY.value] = expiry_dt.date()

            # NOTE use now if ticker.time is None
            observed_dt = effective_date if ticker.time is None else ticker.time

            # NOTE some rows have no expiry date due IBKR not having contract definitions for them
            # we skip these DTE calculations and later drop the rows
            try:
                row[FacetCoreDfColumns.DTE.value] = (
                    expiry_dt.date() - observed_dt.date()
                ).days
            except Exception:
                row[FacetCoreDfColumns.DTE.value] = None

            # row[FacetCoreDfColumns.RIGHT.value] = ticker.contract.right
            row[FacetCoreDfColumns.CALL_PUT.value] = ticker.contract.right.lower()[0]
            row[FacetCoreDfColumns.MULTIPLIER.value] = ticker.contract.multiplier
            row[FacetCoreDfColumns.VOLUME.value] = ticker.volume
            row[FacetCoreDfColumns.SOURCE_ID.value] = ticker.contract.conId

            mg = ticker.modelGreeks
            bg = ticker.bidGreeks
            ag = ticker.askGreeks
            row[FacetCoreDfColumns.DELTA.value] = None if mg is None else mg.delta
            row[FacetCoreDfColumns.VEGA.value] = None if mg is None else mg.vega
            row[FacetCoreDfColumns.GAMMA.value] = None if mg is None else mg.gamma
            row[FacetCoreDfColumns.THETA.value] = None if mg is None else mg.theta
            row[FacetCoreDfColumns.PRICE_MODEL.value] = (
                None if mg is None else mg.optPrice
            )

            row[FacetCoreDfColumns.IV.value] = None if mg is None else mg.impliedVol
            row[FacetCoreDfColumns.IV_ASK.value] = None if ag is None else ag.impliedVol
            row[FacetCoreDfColumns.IV_BID.value] = None if bg is None else bg.impliedVol
            row[FacetCoreDfColumns.UNDERLYING_PRICE.value] = (
                None if mg is None else mg.undPrice
            )

            rows.append(row)

        df = pd.DataFrame(rows)
        df = df.dropna(subset=[FacetCoreDfColumns.DTE.value])
        df.sort_values(
            by=[
                FacetCoreDfColumns.CALL_PUT.value,
                FacetCoreDfColumns.EXPIRY.value,
                FacetCoreDfColumns.STRIKE.value,
            ]
        )
        df.reset_index(drop=True)
        df.iv_ask = df.iv_ask.fillna(np.nan)
        df.iv_bid = df.iv_bid.fillna(np.nan)
        return df


def save(gd: IbkrFutureOptionData, df: pd.DataFrame):
    """
    Save gd/df to the ibkr cache.

    ```
    save(gd, gd.v.enrich_option_tickers_df_with_price)
    ```

    Args:
        gd (IbkrFutureOptionData): _description_
        df (pd.DataFrame): _description_

    Returns:
        _type_: _description_
    """
    df = df[df.edt.notnull()].copy()

    utc_now = datetime.now(pytz.utc)
    mt_now = utc_now.astimezone(pytz.timezone("US/Mountain"))
    data_effective_date = calc_effective_date_of_dt(mt_now)
    # data_effective_date_str: str = data_effective_date.strftime("%Y-%m-%d")

    assert len(df.symbol.unique()) == 1
    ticker = df.symbol.unique()[0]

    # NOTE
    # _ibkr_cache_write - writes a table called ibkr_option_core_facet, in a database called option_pricer_dev
    # f"postgresql+psycopg2://postgres:@localhost:5432/option_pricer_dev"
    # if you need to change the db connection string, see finx_optionpricer/etl/db_ops.py, gen_engine()

    return InstrumentMultiplexer._ibkr_cache_write(
        asset_class=gd.v.asset_category,
        ticker=ticker,
        effective_date=data_effective_date,
        df=df,
    )

In [6]:
def command_execute_graph_download(
    ib: IB,
    market_data_type: MarketDataType = MarketDataType.LIVE,
    symbol: str = "GC",
    dte_max: int = 30,
    pre_fetch_option_filters: List[dict] = None,
):
    """
    Download futures option data for a given underlying symbol.

    Steps:
    1. initialize the graph data object
    2. initialize the underlying symbol and pre-fetch filters
    3. initialize the ibkr downloader
    4. fetch the option chain for the underlying
    5. compute the option chain df
    6. fetch the underlying df
    7. enrich the option chain df with the underlying tickers
    8. apply fetch filters
    9. generate qualified futures options
    10. fetch tickers for qualified futures options
    11. generate option tickers df
    12. enrich option tickers df with the underlying price (because IBKR will drop the underlying price)

    Example,
    ```
    pre_fetch_option_filters_gc = [
        {"moneyness_gte": 95.0},
        {"moneyness_lte": 105.0},
        {"strike_modulus_eq": 10.0},
    ]
    gd = command_execute_graph_download(
        ib=ib,
        symbol="GC",
        dte_max=30,
        pre_fetch_option_filters=pre_fetch_option_filters_gc
    )
    save(gd, gd.v.enrich_option_tickers_df_with_price)
    ```

    Args:
        ib (IB): ib_insync IB object
        symbol (str, optional): underlying symbol. Defaults to "GC".
        dte_max (int, optional): max DTE. Defaults to 30.
        pre_fetch_option_filters (List[dict], optional): pre-fetch option filters. Defaults to None.
        existing_graph_data (IbkrFutureOptionData, optional): existing graph data object. Defaults to None.

    Returns:
        IbkrFutureOptionData: graph data object
    """
    # graph data object
    gd = IbkrFutureOptionData()

    # initialize the underlying symbol and pre-fetch filters
    gd.add_node("underlying_symbol", value=symbol)
    gd.add_node(
        name="pre_fetch_option_filters",
        value=pre_fetch_option_filters,
    )
    gd.add_node("dte_max", value=dte_max)
    gd.compute_all()

    # initialize the ibkr downloader
    ibkr_downloader = IbkrDownloader(ib, market_data_type=market_data_type)

    # fetch the option chain for the underlying
    raw_option_chain = ibkr_downloader.fetch_option_chain_for_underlying(symbol)
    gd.add_node("raw_option_chain", value=raw_option_chain)
    gd.compute("option_chain_df")

    # compute the option chain df
    gd.compute_all()

    xdf = gd.v.filter_option_chain_by_max_expiry_date
    underlying_df = ibkr_downloader.fetch_underlying_df(option_chain_df=xdf)
    gd.add_node("underlying_df", value=underlying_df)
    gd.compute("underlying_df_calc_price")
    gd.compute("enrich_option_chain_df_with_underlying_tickers")

    # apply fetch filters so only get the desired futures options
    gd.compute("apply_fetch_filters")

    qualified_futures_options = ibkr_downloader.gen_qualified_futures_options(
        option_chain_df=gd.v.apply_fetch_filters
    )
    gd.add_node("qualified_futures_options", value=qualified_futures_options)

    futures_option_tickers = ibkr_downloader.fetch_tickers_for_contracts(
        qualified_futures_options
    )
    gd.add_node("futures_option_tickers", value=futures_option_tickers)
    df = ibkr_downloader.gen_option_tickers_df(tickers=futures_option_tickers)
    gd.add_node("option_tickers_df", value=df)

    gd.compute("gen_qualified_fops_df")

    gd.compute("enrich_option_tickers_df_with_price")

    return gd

In [7]:
fetch_option_filters_es = [
    {"moneyness_gte": 98.0},
    {"moneyness_lte": 102.0},
    {"strike_modulus_eq": 50.0},
]
gd = command_execute_graph_download(
    ib=ib,
    symbol="ES",
    dte_max=21,
    pre_fetch_option_filters=fetch_option_filters_es,
    # use frozen market data when market is closed
    market_data_type=MarketDataType.FROZEN,
    # use live market data when market is open
    # market_data_type=MarketDataType.LIVE
)

save(gd, gd.v.enrich_option_tickers_df_with_price)

ibkr=fetch-tickers-1: 100%|██████████| 1/1 [00:11<00:00, 11.05s/it]
2024-10-12 14:46:48.252 | DEBUG    | __main__:apply_fetch_filters:111 - Applying pre fetch option filters: [{'moneyness_gte': 98.0}, {'moneyness_lte': 102.0}, {'strike_modulus_eq': 50.0}]
2024-10-12 14:46:48.252 | DEBUG    | __main__:apply_fetch_filters:112 - Pre Filters,  len => 3639
2024-10-12 14:46:48.255 | DEBUG    | __main__:apply_fetch_filters:125 - Post Filters, len => 75 (98% reduction)


ibkr=fetch-tickers-150: 100%|██████████| 1/1 [00:16<00:00, 16.05s/it]


'wrote ES to ibkr_option_core_facet with 150 rows'

In [46]:
pre_fetch_option_filters_cl = [
    {"moneyness_gte": 95.0},
    {"moneyness_lte": 105.0},
    {"strike_modulus_eq": 5.0},
]
gd = command_execute_graph_download(
    ib=ib,
    symbol="CL",
    dte_max=90,
    pre_fetch_option_filters=pre_fetch_option_filters_cl,
    market_data_type=MarketDataType.FROZEN,
)
save(gd, gd.v.enrich_option_tickers_df_with_price)

ibkr=fetch-tickers-3: 100%|██████████| 1/1 [00:12<00:00, 12.05s/it]
2024-10-12 14:05:07.994 | DEBUG    | __main__:apply_fetch_filters:111 - Applying pre fetch option filters: [{'moneyness_gte': 95.0}, {'moneyness_lte': 105.0}, {'strike_modulus_eq': 5.0}]
2024-10-12 14:05:07.994 | DEBUG    | __main__:apply_fetch_filters:112 - Pre Filters,  len => 2994
2024-10-12 14:05:07.997 | DEBUG    | __main__:apply_fetch_filters:125 - Post Filters, len => 23 (99% reduction)


ibkr=fetch-tickers-46: 100%|██████████| 1/1 [00:13<00:00, 13.06s/it]


'wrote CL to ibkr_option_core_facet with 46 rows'

In [34]:
pre_fetch_option_filters_gc = [
    {"moneyness_gte": 95.0},
    {"moneyness_lte": 105.0},
    {"strike_modulus_eq": 10.0},
]
gd = command_execute_graph_download(
    ib=ib,
    symbol="GC",
    dte_max=90,
    pre_fetch_option_filters=pre_fetch_option_filters_gc,
    market_data_type=MarketDataType.FROZEN,
)
save(gd, gd.v.enrich_option_tickers_df_with_price)

array([304037501, 212921531, 304037391])